In [19]:
!apt update -y
!apt install gcc -y


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
43 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [18]:
# Step 1: Write the code to a .c file
%%writefile parallel_bfs_dfs.c
#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

#define MAX_NODES 100

int graph[MAX_NODES][MAX_NODES];
int visited[MAX_NODES];
int queue[MAX_NODES];
int front = 0, rear = -1;

void enqueue(int node) {
    queue[++rear] = node;
}

int dequeue() {
    return queue[front++];
}

int isQueueEmpty() {
    return front > rear;
}

void parallelBFS(int start, int n) {
    enqueue(start);
    visited[start] = 1;

    while (!isQueueEmpty()) {
        int currentSize = rear - front + 1;

        #pragma omp parallel for shared(queue, graph, visited, currentSize)
        for (int i = 0; i < currentSize; i++) {
            int node;

            #pragma omp critical
            {
                node = dequeue();
                printf("Visited %d by thread %d\n", node, omp_get_thread_num());
            }

            for (int j = 0; j < n; j++) {
                if (graph[node][j] && !visited[j]) {
                    #pragma omp critical
                    {
                        if (!visited[j]) {
                            visited[j] = 1;
                            enqueue(j);
                        }
                    }
                }
            }
        }
    }
}

void dfsTask(int node, int n) {
    int shouldProceed = 0;

    #pragma omp critical
    {
        if (!visited[node]) {
            visited[node] = 1;
            printf("Visited %d by thread %d\n", node, omp_get_thread_num());
            shouldProceed = 1;
        }
    }

    if (!shouldProceed) return;

    for (int i = 0; i < n; i++) {
        if (graph[node][i]) {
            #pragma omp task
            dfsTask(i, n);
        }
    }

    #pragma omp taskwait
}

void parallelDFS(int start, int n) {
    #pragma omp parallel
    {
        #pragma omp single
        {
            dfsTask(start, n);
        }
    }
}

int main() {
    int n = 6;

    int adjacencyMatrix[6][6] = {
        {0, 1, 1, 0, 0, 0},
        {1, 0, 0, 1, 1, 0},
        {1, 0, 0, 0, 1, 0},
        {0, 1, 0, 0, 0, 1},
        {0, 1, 1, 0, 0, 1},
        {0, 0, 0, 1, 1, 0}
    };

    for (int i = 0; i < n; i++)
        for (int j = 0; j < n; j++)
            graph[i][j] = adjacencyMatrix[i][j];

    printf("Parallel BFS:\n");
    for (int i = 0; i < n; i++) visited[i] = 0;
    front = 0, rear = -1;
    parallelBFS(0, n);

    printf("\nParallel DFS:\n");
    for (int i = 0; i < n; i++) visited[i] = 0;
    parallelDFS(0, n);

    return 0;
}


Overwriting parallel_bfs_dfs.c


In [20]:
!gcc -fopenmp parallel_bfs_dfs.c -o parallel_bfs_dfs


In [21]:
!./parallel_bfs_dfs


Parallel BFS:
Visited 0 by thread 0
Visited 1 by thread 1
Visited 2 by thread 0
Visited 3 by thread 0
Visited 4 by thread 1
Visited 5 by thread 0

Parallel DFS:
Visited 0 by thread 0
Visited 1 by thread 1
Visited 2 by thread 0
Visited 4 by thread 0
Visited 5 by thread 0
Visited 3 by thread 0
